# Preamble

In [1]:
%run ./spark-instance.ipynb

SparkConf created
Started SparkSession
Spark version 3.3.1


In [15]:
clean_spark()

CLEANING SPARK INSTANCE...


In [12]:
from datetime import datetime

import pandas as pd

# Definitions

In [5]:
ldap_root_dse_raw_base_path_fmt = "s3a://catrin/measurements/tool=goscanner/format=raw/port={port}/scan=ldap_metadata/result=ldap_root_dse_raw/"
ldap_root_dse_raw_path_fmt = "s3a://catrin/measurements/tool=goscanner/format=raw/port={port}/scan=ldap_metadata/result=ldap_root_dse_raw/year={year}/month={month:02d}/day={day:02d}"

ports = ["389", "636", "3268", "3269"]

# Retrieve scanning dates

In [10]:
pdf = pd.DataFrame()
for port in ports:
    p = ldap_root_dse_raw_base_path_fmt.format(port=port)
    
    df = spark.read.option("header", "true"
                  ).option("multiline", "true"
                  ).option("wholeFile", "true"
                  ).option("inferSchema", "true"
                  ).option("basePath", p
                  ).csv(p)
    _pdf = df.select("year", "month", "day").dropDuplicates(["year", "month", "day"]).toPandas()
    _pdf["port"] = pd.to_numeric(port)
    pdf = pd.concat([pdf, _pdf])
    break